# Density-Based Clustering
Alex Gajowski

In [1]:
epsilon = 0.6
eta = 0.7
mu = 12

In [2]:
import csv

In [3]:
class Vert:
    def __init__(self):
        self.edges = dict()
        self.attrs = dict()
        
    def __repr__(self):
        return 'edges:' + str(self.edges) + ', attrs:' + str(self.attrs)

In [61]:
import mysql.connector

mydb = mysql.connector.connect(
    host='localhost',
    user='alex',
    database='cora'
)

cursor = mydb.cursor()

In [10]:
colorMap = { 'Genetic_Algorithms': 'red',
'Reinforcement_Learning': 'orange',
'Theory': 'yellow',
'Rule_Learning': 'green',
'Case_Based': 'blue',
'Probabilistic_Methods': 'purple',
'Neural_Networks': 'pink',
}

In [4]:
G = dict()
edgeWeights = list()

with open('cora_edges.csv') as edgesFile:
    coraEdges = csv.reader(edgesFile)
    next(coraEdges)
    for e in coraEdges:
        i = e[0]
        j = e[1]
        w = float(e[2])**2
                
        if w >= 1:
            continue
        edgeWeights.append(w)

        
        if not i in G:
            G[i] = Vert()
            
        if not j in G:
            G[j] = Vert()     
        
        if not j in G[i].edges:
            G[i].edges[j] = w
        
        if not i in G[j].edges:
            G[j].edges[i] = w


#G


In [5]:
def commonNeighbors(u, v):
    n_u = set(G[u].edges.keys())
    n_v = set(G[v].edges.keys())

    return len(n_u.intersection(n_v)) / len(n_u.union(n_v))

In [6]:
def getNextNode(cluster, clNeighbors):
    edgeWeightSums = [(e[0], sum(G[e[0]].edges[o] for o in e[1])) for e in [(n, set(cluster).intersection(set(G[n].edges.keys()))) for n in clNeighbors ]]
    sortedEdgeWeightSums = sorted(edgeWeightSums, key=lambda e: e[1], reverse=True)
    topEdgesByWeight = list(filter(lambda x: x[1] == sortedEdgeWeightSums[0][1], sortedEdgeWeightSums))
    # len(set(G[t[0]].edges.keys()).difference(cluster).intersection(clNeighbors)), 
    return sorted([(t[0], t[1], len(set(G[t[0]].edges.keys()).intersection( cluster.union(clNeighbors) )) / len(G[t[0]].edges.keys())) for t in topEdgesByWeight], key=lambda e: e[2], reverse=True)[0]

In [7]:
#cl = set()
#cl.add('35')
#cl.add('1050679')

#clN = set()
#clN.update([x for x in G['35'].edges.keys()])
#clN.update([x for x in G['1050679'].edges.keys()])

#getNextNode(cl, clN)

In [8]:
#sorted([(x[0], len(x[1].edges)) for x in G.items()],key=lambda y: y[1], reverse=True)

In [9]:
def densityCluster(t_w):
    # 5 - 7
    nodeExists = dict()
    for node, edges_node in G.items():
        nodeExists[node] = True
    
    # 8
    clusters = list()
    
    # 9
    nodeDegree = dict()
    # 10 - 12
    for node, edges_node in G.items():
        nodeDegree[node] = len(edges_node.edges)
        
    
    for nId, __n_edges in sorted([(x[0], len(x[1].edges)) for x in G.items()],key=lambda y: y[1], reverse=True):
        node = str(nId)
        if nodeExists[node]:
            c = set()
            maxDegree = node
            nodeDegree[maxDegree] = -1
            nodeExists[maxDegree] = False
            neighbors = set(G[maxDegree].edges.keys())
            c.add(maxDegree)
            
            #for neighbor, edge_weight in neighbors:
                #print(float(edge_weight) + commonNeighbors(node, neighbor))
            
            #print(neighbors)
            
            while len(neighbors) > 0:
                neighbor, edge_weight, no_neighbors = getNextNode(c, neighbors)
                neighbors.remove(neighbor)
                
                if not nodeExists[neighbor]:
                    continue
                
                #print(no_neighbors)
                
                if (edge_weight + no_neighbors) >= t_w:
                    c.add(neighbor)
                    nodeExists[neighbor] = False
                    nodeDegree[neighbor] = -1
                    
                    newNeighbors = list(filter(lambda x: nodeExists[x], G[neighbor].edges.keys()))
                    neighbors.update( newNeighbors )
                                    
            #print(neighbors)
            
            clusters.append(c)
            
    return clusters


In [60]:
T_W = 0.3
cs = sorted(densityCluster(T_W), key=lambda x: len(x), reverse=True)

In [62]:
coloredVerts = {}
if True:
    for cl in cs:
        if len(cl) < 5:
            continue
    
        topicCounts = dict()
        for pid in cl:
            cursor.execute("SELECT class_label FROM paper WHERE paper_id="+pid)
            topic = next(cursor)[0]
            
        
            coloredVerts[pid] = colorMap[topic]
        
            if not topic in topicCounts:
                topicCounts[topic] = 0
            topicCounts[topic] += 1
        
        cTopics = sorted(topicCounts.items(), key=lambda x: x[1], reverse=True)
    
        print('Size: ' + str(len(cl)))
        for cTopic in cTopics:
            topicPercent = cTopic[1]/len(cl) * 100
            percent = str(round(topicPercent, 1))
            print(percent + '% ' + cTopic[0] + ' (' + str(cTopic[1]) + ')')

        for u in cl:
            coloredVerts[u] = colorMap[cTopics[0][0]]
        
        print()
else:
    for pid in G.keys():
        cursor.execute("SELECT class_label FROM paper WHERE paper_id="+pid)
        topic = next(cursor)[0]
        coloredVerts[pid] = colorMap[topic]

Size: 2193
28.8% Neural_Networks (632)
17.9% Genetic_Algorithms (392)
14.4% Theory (316)
12.6% Case_Based (277)
10.9% Probabilistic_Methods (239)
9.5% Reinforcement_Learning (209)
5.8% Rule_Learning (128)

Size: 54
100.0% Probabilistic_Methods (54)

Size: 28
100.0% Probabilistic_Methods (28)

Size: 22
100.0% Probabilistic_Methods (22)

Size: 18
100.0% Rule_Learning (18)

Size: 18
100.0% Neural_Networks (18)

Size: 14
85.7% Neural_Networks (12)
14.3% Probabilistic_Methods (2)

Size: 12
100.0% Probabilistic_Methods (12)

Size: 9
100.0% Probabilistic_Methods (9)

Size: 8
100.0% Rule_Learning (8)

Size: 8
100.0% Rule_Learning (8)

Size: 7
85.7% Theory (6)
14.3% Neural_Networks (1)

Size: 7
100.0% Neural_Networks (7)

Size: 6
100.0% Neural_Networks (6)

Size: 5
80.0% Theory (4)
20.0% Neural_Networks (1)

Size: 5
100.0% Probabilistic_Methods (5)

Size: 5
100.0% Theory (5)

Size: 5
60.0% Case_Based (3)
40.0% Genetic_Algorithms (2)



In [63]:
#"""
from pyvis.network import Network
import pandas as pd

cora_net = Network(height="100%", width="100%", bgcolor="#222222", font_color="white")
#cora_net.prep_notebook()

#cora_net.set_options(" ""
#{
#  "nodes": {
#    "fixed": {
#      "x": true,
#      "y": true
#    }
#  }
#}
#" "")

#cora_net.toggle_physics(False)

# set the physics layout of the network
#cora_net.barnes_hut()
#print(cora_data)

for src in G.keys():
    
    src_color = 'gray'
    if src in coloredVerts:
        src_color = coloredVerts[src]
    #else:
    #    continue
    
    cora_net.add_node(src, label=" ", title=src, color=src_color)

    for dst, w in G[src].edges.items():
        dst_color = 'gray'
        if dst in coloredVerts:
            dst_color = coloredVerts[dst]
        #else:
        #    continue
        
        cora_net.add_node(dst, label=" ", title=dst, color=src_color)

        
        cora_net.add_edge(src, dst, value=w) #, physics=False)

for e in cora_net.edges:
    e['title'] = e['value']
            
cora_net.show_buttons(filter_=['nodes'])

cora_net.save_graph("cora" + str(T_W) + ".html")
#"""

In [59]:
#coloredVerts.keys()

In [16]:
sorted(edgeWeights, reverse=True)

[0.9111570247933886,
 0.9111570247933886,
 0.8975069252077561,
 0.8975069252077561,
 0.7744,
 0.7159763313609467,
 0.7055999999999999,
 0.7055999999999999,
 0.6944444444444445,
 0.6824196597353497,
 0.6824196597353497,
 0.655328798185941,
 0.6523668639053255,
 0.626736111111111,
 0.626736111111111,
 0.5804988662131518,
 0.5804988662131518,
 0.5340236686390532,
 0.5017361111111112,
 0.48999999999999994,
 0.4792899408284023,
 0.4792899408284023,
 0.4444444444444444,
 0.4444444444444444,
 0.42925089179548154,
 0.42533081285444235,
 0.41326530612244905,
 0.41326530612244905,
 0.4049586776859504,
 0.390625,
 0.38525564803804996,
 0.37869822485207105,
 0.37051039697542537,
 0.3686224489795918,
 0.3686224489795918,
 0.34027777777777785,
 0.33284023668639046,
 0.33284023668639046,
 0.308641975308642,
 0.28994082840236685,
 0.2722117202268431,
 0.2688614540466392,
 0.25,
 0.25,
 0.25,
 0.25,
 0.25,
 0.25,
 0.25,
 0.25,
 0.2330558858501784,
 0.2304,
 0.2304,
 0.2304,
 0.2177777777777778,
 0.2177

In [74]:
tws = [i/10 for i in range(0, 12)]
twClustering = [(tw, sorted(densityCluster(tw), key=lambda x: len(x), reverse=True)) for tw in tws]

In [75]:
for v in G.keys():
    cursor.execute("SELECT class_label FROM paper WHERE paper_id="+v)
    topic = next(cursor)[0]
    G[v].attrs['topic'] = topic        

In [80]:
qualityEvalThresh = 5

def clusterQuality(clustering): 
    correctlyLabeled = 0
    wronglyLabeled = 0
    
    for cluster in clustering:
        if len(cluster) < 5:
            continue
        
        topicsInCluster = {}
        for paper in cluster:
            if not G[paper].attrs['topic'] in topicsInCluster:
                topicsInCluster[G[paper].attrs['topic']] = 0
            topicsInCluster[G[paper].attrs['topic']] += 1
        
        topicsSorted = sorted(topicsInCluster.items(), key=lambda x: x[1], reverse=True)
        correctlyLabeled += topicsSorted[0][1]
        for topic, mislabeled in topicsSorted[1:]:
            wronglyLabeled += mislabeled
    
    return (correctlyLabeled, wronglyLabeled)

In [82]:
clusterRatios = [(tw, clusterQuality(clustering)) for tw, clustering in twClustering]

In [83]:
clusterRatios

[(0.0, (727, 1705)),
 (0.1, (739, 1693)),
 (0.2, (767, 1665)),
 (0.3, (857, 1567)),
 (0.4, (1688, 290)),
 (0.5, (1520, 246)),
 (0.6, (845, 60)),
 (0.7, (467, 23)),
 (0.8, (275, 20)),
 (0.9, (161, 13)),
 (1.0, (132, 12)),
 (1.1, (0, 0))]

In [102]:
for tw, quality in clusterRatios:
    correct, wrong = quality
    perCorr = str(round((100*correct/len(G)), 3))
    perWrong = str(round((100*wrong/len(G)), 3))
    perQuality = str(round(( 100*(correct-wrong)/len(G)), 3))
    print(str(tw) + '\t' + str(correct) + '\t'+ perCorr + '\t' + str(wrong) + '\t' + perWrong + '\t' + perQuality)

0.0	727	27.716	1705	65.002	-37.286
0.1	739	28.174	1693	64.544	-36.371
0.2	767	29.241	1665	63.477	-34.236
0.3	857	32.673	1567	59.741	-27.068
0.4	1688	64.354	290	11.056	53.298
0.5	1520	57.949	246	9.379	48.57
0.6	845	32.215	60	2.287	29.928
0.7	467	17.804	23	0.877	16.927
0.8	275	10.484	20	0.762	9.722
0.9	161	6.138	13	0.496	5.642
1.0	132	5.032	12	0.457	4.575
1.1	0	0.0	0	0.0	0.0
